In [1]:
import os
os.chdir('../')

In [2]:
pwd

'/Users/hitam/Downloads/End2End_ChickenDiseaseClassification'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/subramanyanayak3/End2End_ChickenDiseaseClassification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="subramanyanayak3"
os.environ["MLFLOW_TRACKING_PASSWORD"]="b18ad68cf0c678f5f616a47081e6c4c129e4a844"

In [4]:
import os
print("MLFLOW_TRACKING_URI:", os.environ.get("MLFLOW_TRACKING_URI"))
print("MLFLOW_TRACKING_USERNAME:", os.environ.get("MLFLOW_TRACKING_USERNAME"))
print("MLFLOW_TRACKING_PASSWORD:", os.environ.get("MLFLOW_TRACKING_PASSWORD"))

MLFLOW_TRACKING_URI: https://dagshub.com/subramanyanayak3/End2End_ChickenDiseaseClassification.mlflow
MLFLOW_TRACKING_USERNAME: subramanyanayak3
MLFLOW_TRACKING_PASSWORD: b18ad68cf0c678f5f616a47081e6c4c129e4a844


In [5]:
import tensorflow as tf

2024-01-29 16:58:22.468639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    mlflow_uri: str

In [8]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import *

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            mlflow_uri = 'https://dagshub.com/subramanyanayak3/End2End_ChickenDiseaseClassification.mlflow'
        )
        return eval_config

In [10]:
from urllib.parse import urlparse
import mlflow
import mlflow.tensorflow
import mlflow.sklearn

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        self.tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        with mlflow.start_run():
            self.log_into_mlflow()
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)


            mlflow.log_params(self.config.all_params)


            
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            


            # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                

            # Log metrics
            if self.tracking_url_type_store != "file":

                # Register the model
                
                mlflow.sklearn.log_model(model, "model", registered_model_name="ChickenDisease")
            else:
                mlflow.sklearn.log_model(model, "model")


    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

In [14]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e


[2024-01-29 17:06:14,864: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-29 17:06:14,876: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-29 17:06:14,880: INFO: common: created directory at: artifacts]


Found 116 images belonging to 2 classes.
8/8 [==============================] - 51s 6s/step - loss: 1.2388 - accuracy: 0.7586


/Users/hitam/Downloads/End2End_ChickenDiseaseClassification/env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ChickenDisease'.
2024/01/29 17:13:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ChickenDisease, version 1
Created version '1' of model 'ChickenDisease'.


[2024-01-29 17:13:07,324: INFO: common: json file saved at: scores.json]
